In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
import seaborn as sns
import matplotlib.pyplot as plt
from pigeon import annotate
#import ipywidgets as widgets
#from ipywidgets import interactive

In [2]:
labels_df = pd.read_csv(join("data", "cis-pd", "data_labels", "CIS-PD_Training_Data_IDs_Labels.csv"))
labels_df = labels_df.sort_values(by="tremor", ascending=False).reset_index(drop=True)

In [3]:
labels_df.head()

,measurement_id,subject_id,on_off,dyskinesia,tremor
0,4913062f-b696-40c6-a0f2-87efe1dd4810,1048,3.0,3.0,4.0
1,7b2698a6-f3a5-4345-9630-25d377d58992,1048,2.0,3.0,4.0
2,40f713fe-0ac9-4cd0-809c-0a8024955a2a,1048,3.0,3.0,4.0
3,e0441156-c4b8-467c-8f4f-3b532d594d8f,1048,4.0,3.0,4.0
4,daf11494-e6fa-4376-a78a-86c683885764,1004,4.0,4.0,4.0


In [4]:
m_id = "4913062f-b696-40c6-a0f2-87efe1dd4810" # 4 tremor

m_df = pd.read_csv(join("data", "cis-pd", "training_data", f"{m_id}.csv"))
m_df.head()

,Timestamp,X,Y,Z
0,0.00,-0.159668,1.001221,0.031006
1,0.02,-0.148438,0.989746,0.077148
2,0.04,-0.154297,0.990723,0.081055
3,0.06,-0.158203,0.990723,0.073975
4,0.08,-0.157227,0.976562,0.084229


In [5]:
interval_size = 0.5

In [6]:
m_molten_df = m_df.melt(id_vars=["Timestamp"], value_vars=["X", "Y", "Z"], var_name="dim")
m_molten_df.head()

,Timestamp,dim,value
0,0.00,X,-0.159668
1,0.02,X,-0.148438
2,0.04,X,-0.154297
3,0.06,X,-0.158203
4,0.08,X,-0.157227


In [7]:
interval_start = 0.0
interval_min = m_df["Timestamp"].min()
interval_max = m_df["Timestamp"].max()

In [8]:
def display_interval(interval_start):
    interval_end = interval_start + interval_size
    extended_interval_molten_df = m_molten_df.loc[(m_molten_df["Timestamp"] >= interval_start - interval_size) & (m_molten_df["Timestamp"] < interval_end + interval_size)]
    left_molten_df = extended_interval_molten_df.loc[(extended_interval_molten_df["Timestamp"] >= interval_start - interval_size) & (extended_interval_molten_df["Timestamp"] < interval_start)]
    interval_molten_df = extended_interval_molten_df.loc[(extended_interval_molten_df["Timestamp"] >= interval_start) & (extended_interval_molten_df["Timestamp"] < interval_end)]
    right_molten_df = extended_interval_molten_df.loc[(extended_interval_molten_df["Timestamp"] >= interval_end) & (extended_interval_molten_df["Timestamp"] < interval_end + interval_size)]
    
    fig, axes = plt.subplots(1, 3, sharey=True, figsize=(14, 5))

    ax = sns.lineplot(x="Timestamp", y="value", hue="dim", ax=axes[0], data=left_molten_df)
    ax.set_axis_off()
    ax.get_legend().remove()
    ax = sns.lineplot(x="Timestamp", y="value", hue="dim", ax=axes[1], data=interval_molten_df)
    ax = sns.lineplot(x="Timestamp", y="value", hue="dim", ax=axes[2], data=right_molten_df)
    ax.set_axis_off()
    ax.get_legend().remove()
    plt.ylim(-1.3, 1.3)
    plt.show()

In [9]:
annotations = annotate(
  np.arange(interval_min, interval_max, interval_size),
  options=['tremor', 'no_tremor'],
  display_fn=lambda interval_start: display_interval(interval_start)
)

HTML(value='0 examples annotated, 1601 examples left')

Output()

In [10]:
annotations

[]

In [11]:
a_dicts = [ { "interval_start": x[0], "annotation": x[1] } for x in annotations ]

In [12]:
a_df = pd.DataFrame(data=a_dicts)
a_df.head()

""


In [13]:
if a_df.shape[0] >= 100:
    a_df.to_csv(join("data", "cis-pd", "training_data_tremor_annotations", f"{m_id}.csv"))